In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!pip install scanpy --quiet

     |████████████████████████████████| 2.0 MB 5.4 MB/s 
     |████████████████████████████████| 96 kB 4.4 MB/s 
     |████████████████████████████████| 88 kB 5.9 MB/s 
     |████████████████████████████████| 9.4 MB 33.3 MB/s 
     |████████████████████████████████| 295 kB 64.8 MB/s 
     |████████████████████████████████| 965 kB 69.9 MB/s 
     |████████████████████████████████| 1.1 MB 48.6 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 


In [3]:
import random
import torch
import sys
import os
import gc
import collections 
import anndata as ad
import torch.nn as nn 
from argparse import Namespace

config = Namespace(
    # LEARNING_RATE = 0.00002,
    DEVICE = 'cpu',
    BATCH_SIZE = 100,
    NUM_WORKERS = 4,
    N_GENES = 13431,
    N_PEAKS = 116465,
    MAX_SEQ_LEN_GEX = 1500,
    MAX_SEQ_LEN_ATAC = 15000,
)

In [4]:
execfile("drive/MyDrive/Colab_Notebooks/CPSC532S/final_project/code/resources/data.py")
execfile("drive/MyDrive/Colab_Notebooks/CPSC532S/final_project/code/resources/models.py")

## Load data

In [5]:
index = get_chr_index(ad.read_h5ad("drive/MyDrive/Colab_Notebooks/CPSC532S/final_project/data/ATAC_processed.h5ad"))

In [6]:
gc.collect()

307

In [7]:
batch = ad.read_h5ad("drive/MyDrive/Colab_Notebooks/CPSC532S/final_project/data/GEX_processed.h5ad").obs['batch']
batch = list(batch)
# train_id = [a for a, l in enumerate(batch) if l not in ['s2d4','s1d1']]
val_id =  [a for a, l in enumerate(batch) if l == 's1d1']
# test_id = [a for a, l in enumerate(batch) if l == 's2d4']

In [8]:
del batch
gc.collect()

279

In [9]:
cell_type_all = ad.read_h5ad("drive/MyDrive/Colab_Notebooks/CPSC532S/final_project/data/GEX_processed.h5ad").obs['cell_type']
gc.collect()
csr_gex = ad.read_h5ad("drive/MyDrive/Colab_Notebooks/CPSC532S/final_project/data/GEX_processed.h5ad").layers['log_norm']
gc.collect()
csr_atac = ad.read_h5ad("drive/MyDrive/Colab_Notebooks/CPSC532S/final_project/data/ATAC_processed.h5ad").layers['log_norm']
gc.collect()

242

In [10]:
random.seed(0)

# idx_val = [val_id[i] for i in random.sample(range(0, 6000), 1024)]
idx_val = val_id  # Full dataset
gex_val = csr_gex[idx_val,:]
atac_val = csr_atac[idx_val,:]
cell_type_val = [cell_type_all[j] for j in idx_val]

data_val = get_dataloaders(gex_val, atac_val, cell_type_val)

In [18]:
def cellTypematchingProb2(gex_out_0, gex_out_1, atac_out_0, atac_out_1, cell_type):
        gex_out_0 = nn.functional.normalize(gex_out_0, dim = 1)
        gex_out_1 = nn.functional.normalize(gex_out_1, dim = 1)
        atac_out_0 = nn.functional.normalize(atac_out_0, dim = 1)
        atac_out_1 = nn.functional.normalize(atac_out_1, dim = 1)
        
        score_mat = torch.mm(gex_out_0, atac_out_0.transpose(0,1)) + 0.2 * torch.mm(gex_out_1, atac_out_1.transpose(0,1))

        #cell matching prob
        score_norm_gex = score_mat.softmax(dim = 0)
        score_norm_atac = score_mat.softmax(dim = 1)

        match_probs = 0.5 * (torch.diagonal(score_norm_gex) + torch.diagonal(score_norm_atac))

      # method1: xf
      # Collect list of index list for each cell type
        idx_in_type = collections.defaultdict(list)
        for i, x in enumerate(cell_type):
            idx_in_type[x].append(i)

        sum_score_mat=torch.zeros(len(idx_in_type.values()),len(idx_in_type.values()))
        for i, dx in enumerate(idx_in_type.values()):
          for j, dx2 in enumerate(idx_in_type.values()):
            tem=score_mat[np.ix_(dx, dx2)].sum()
            sum_score_mat[i,j]=tem
        score_mat_norm = 0.5 * (sum_score_mat.softmax(dim = 0) + sum_score_mat.softmax(dim = 1))
       
        return torch.mean(torch.diagonal(score_mat_norm)), torch.mean(match_probs) 
        #first and second output is celltype matching prob, last output is cell matching prob

def inference(model, data_val):

    # Initialize encoder & decoder 
    model.eval()
    model.to(config.DEVICE)
    criterion.to(config.DEVICE)
    
    running_loss_cross = 0.0
    running_loss_triplet = 0.0
    running_ct_prob = running_ct_prob2 = 0.0  
    running_cell_prob = running_cell_prob2 = 0.0  
    for iter, data in enumerate(data_val):
      gex_input = data['gex'].to(config.DEVICE)
      atac_input = data['atac'].to(config.DEVICE)
      cell_type_input = data['cell_type']

      ### Forward
      gex_out_0, gex_out_1, atac_out_0, atac_out_1 = model(gex_input, atac_input)

      ### Compute loss
      loss, loss_triplet, loss_cross, ct_match_prob, cell_match_prob = criterion(gex_out_0, gex_out_1, atac_out_0, atac_out_1, cell_type_input)
      ct_match_prob2, cell_match_prob2 = cellTypematchingProb2(gex_out_0, gex_out_1, atac_out_0, atac_out_1, cell_type_input)

      running_loss_cross += loss_cross.item()
      running_loss_triplet += loss_triplet.item()
      running_ct_prob += ct_match_prob.item()
      running_ct_prob2 += ct_match_prob2.item()
      running_cell_prob += cell_match_prob.item()
      running_cell_prob2 += cell_match_prob2.item()

      del gex_input
      del atac_input
      del cell_type_input
      gc.collect()
      # torch.cuda.empty_cache()
      # if (iter + 1) % 10 == 0: print("iter", iter)

    return running_loss_cross / len(data_val), running_loss_triplet / len(data_val), running_ct_prob / len(data_val), running_ct_prob2 / len(data_val), running_cell_prob / len(data_val), running_cell_prob2 / len(data_val)

## Parameter set 1

In [19]:
config.ALPHA = 0.2
config.MARGIN = 0.5
config.N_CHANNELS = 32

Null model:

In [20]:
model = Encoder(kernel_size_gex = 100, kernel_size_atac_1 = 30, kernel_size_atac_2 = 5, index = index).to(config.DEVICE) ## CHANGED TO SMALLER KERNAL SIZE FOR ATAC
criterion = bidirectTripletLoss(alpha = config.ALPHA, margin = config.MARGIN).to('cpu')
loss_cross, loss_triplet, ct_match_prob, ct_match_prob2, cell_match_prob, cell_match_prob2 = inference(model, data_val)
print('cross entropy loss = {0}, triplet loss = {1}, \n NS cell type match prob = {2}, XF cell type match prob = {3}, \n NS cell match prob = {4}, XF cell match prob = {5}'.format(
    loss_cross,
    loss_triplet, 
    ct_match_prob,
    ct_match_prob2,
    cell_match_prob, 
    cell_match_prob2 
    )
)

cross entropy loss = 4.583524030352396, triplet loss = 0.9396619238550701, 
 NS cell type match prob = 0.0606274740208709, XF cell type match prob = 0.08384680499633153, 
 NS cell match prob = 0.010499447303277159, XF cell match prob = 0.010499447303277159


Trained model:

In [21]:
file = 'drive/MyDrive/Colab_Notebooks/CPSC532S/final_project/model/trained_model_semihard_allcells_alpha' + \
        str(config.ALPHA) + '_margin' + str(config.MARGIN) + '_nchannels' + str(config.N_CHANNELS) + '_15epochs'
model.load_state_dict(torch.load(file, map_location=torch.device('cpu')))
loss_cross, loss_triplet, ct_match_prob, ct_match_prob2, cell_match_prob, cell_match_prob2 = inference(model, data_val)

In [22]:
print('cross entropy loss = {0}, triplet loss = {1}, \n NS cell type match prob = {2}, XF cell type match prob = {3}, \n NS cell match prob = {4}, XF cell match prob = {5}'.format(
    loss_cross,
    loss_triplet, 
    ct_match_prob,
    ct_match_prob2,
    cell_match_prob, 
    cell_match_prob2 
    )
)

cross entropy loss = 3.8042790057167175, triplet loss = 0.5601936408451625, 
 NS cell type match prob = 0.12759629045687024, XF cell type match prob = 0.3420316099174439, 
 NS cell match prob = 0.023314881123720654, XF cell match prob = 0.023314881123720654


## Parameter set 2

In [23]:
config.ALPHA = 0.2
config.MARGIN = 0.5
config.N_CHANNELS = 64

Null model:

In [24]:
model = Encoder(kernel_size_gex = 100, kernel_size_atac_1 = 30, kernel_size_atac_2 = 5, index = index).to(config.DEVICE) ## CHANGED TO SMALLER KERNAL SIZE FOR ATAC
criterion = bidirectTripletLoss(alpha = config.ALPHA, margin = config.MARGIN).to('cpu')
loss_cross, loss_triplet, ct_match_prob, ct_match_prob2, cell_match_prob, cell_match_prob2 = inference(model, data_val)
print('cross entropy loss = {0}, triplet loss = {1}, \n NS cell type match prob = {2}, XF cell type match prob = {3}, \n NS cell match prob = {4}, XF cell match prob = {5}'.format(
    loss_cross,
    loss_triplet, 
    ct_match_prob,
    ct_match_prob2,
    cell_match_prob, 
    cell_match_prob2 
    )
)

cross entropy loss = 4.583401907057989, triplet loss = 0.9345084107111371, 
 NS cell type match prob = 0.06064627307748038, XF cell type match prob = 0.06100164714550215, 
 NS cell match prob = 0.010504852477756758, XF cell match prob = 0.010504852477756758


Trained model:

In [25]:
file = 'drive/MyDrive/Colab_Notebooks/CPSC532S/final_project/model/trained_model_semihard_allcells_alpha' + \
        str(config.ALPHA) + '_margin' + str(config.MARGIN) + '_nchannels' + str(config.N_CHANNELS) + '_15epochs'
model.load_state_dict(torch.load(file, map_location=torch.device('cpu')))
loss_cross, loss_triplet, ct_match_prob, ct_match_prob2, cell_match_prob, cell_match_prob2 = inference(model, data_val)
print('cross entropy loss = {0}, triplet loss = {1}, \n NS cell type match prob = {2}, XF cell type match prob = {3}, \n NS cell match prob = {4}, XF cell match prob = {5}'.format(
    loss_cross,
    loss_triplet, 
    ct_match_prob,
    ct_match_prob2,
    cell_match_prob, 
    cell_match_prob2 
    )
)

cross entropy loss = 3.796861069543021, triplet loss = 0.5583841091110593, 
 NS cell type match prob = 0.12720231664559198, XF cell type match prob = 0.38910256207935395, 
 NS cell match prob = 0.023455432867483486, XF cell match prob = 0.023455432867483486


## Parameter set 3

In [26]:
config.ALPHA = 0.2
config.MARGIN = 1
config.N_CHANNELS = 32

Null model:

In [27]:
model = Encoder(kernel_size_gex = 100, kernel_size_atac_1 = 30, kernel_size_atac_2 = 5, index = index).to(config.DEVICE) ## CHANGED TO SMALLER KERNAL SIZE FOR ATAC
criterion = bidirectTripletLoss(alpha = config.ALPHA, margin = config.MARGIN).to('cpu')
loss_cross, loss_triplet, ct_match_prob, ct_match_prob2, cell_match_prob, cell_match_prob2 = inference(model, data_val)
print('cross entropy loss = {0}, triplet loss = {1}, \n NS cell type match prob = {2}, XF cell type match prob = {3}, \n NS cell match prob = {4}, XF cell match prob = {5}'.format(
    loss_cross,
    loss_triplet, 
    ct_match_prob,
    ct_match_prob2,
    cell_match_prob, 
    cell_match_prob2 
    )
)

cross entropy loss = 4.582456770397368, triplet loss = 1.9236042631997003, 
 NS cell type match prob = 0.06065472700293102, XF cell type match prob = 0.08209753249372755, 
 NS cell match prob = 0.010510652444310604, XF cell match prob = 0.010510652444310604


Trained model:

In [28]:
file = 'drive/MyDrive/Colab_Notebooks/CPSC532S/final_project/model/trained_model_semihard_allcells_alpha' + \
        str(config.ALPHA) + '_margin' + str(config.MARGIN) + '_nchannels' + str(config.N_CHANNELS) + '_15epochs'
model.load_state_dict(torch.load(file, map_location=torch.device('cpu')))
loss_cross, loss_triplet, ct_match_prob, ct_match_prob2, cell_match_prob, cell_match_prob2 = inference(model, data_val)
print('cross entropy loss = {0}, triplet loss = {1}, \n NS cell type match prob = {2}, XF cell type match prob = {3}, \n NS cell match prob = {4}, XF cell match prob = {5}'.format(
    loss_cross,
    loss_triplet, 
    ct_match_prob,
    ct_match_prob2,
    cell_match_prob, 
    cell_match_prob2 
    )
)

cross entropy loss = 3.77757455810668, triplet loss = 1.0215108754142883, 
 NS cell type match prob = 0.13271513167354795, XF cell type match prob = 0.32955073553418357, 
 NS cell match prob = 0.023966115884601125, XF cell match prob = 0.023966115884601125


## Parameter set 4

In [29]:
config.ALPHA = 0.2
config.MARGIN = 1
config.N_CHANNELS = 64

Null model:

In [30]:
model = Encoder(kernel_size_gex = 100, kernel_size_atac_1 = 30, kernel_size_atac_2 = 5, index = index).to(config.DEVICE) ## CHANGED TO SMALLER KERNAL SIZE FOR ATAC
criterion = bidirectTripletLoss(alpha = config.ALPHA, margin = config.MARGIN).to('cpu')
loss_cross, loss_triplet, ct_match_prob, ct_match_prob2, cell_match_prob, cell_match_prob2 = inference(model, data_val)
print('cross entropy loss = {0}, triplet loss = {1}, \n NS cell type match prob = {2}, XF cell type match prob = {3}, \n NS cell match prob = {4}, XF cell match prob = {5}'.format(
    loss_cross,
    loss_triplet, 
    ct_match_prob,
    ct_match_prob2,
    cell_match_prob, 
    cell_match_prob2 
    )
)

cross entropy loss = 4.586007080380878, triplet loss = 1.9357188485917591, 
 NS cell type match prob = 0.06053165322731412, XF cell type match prob = 0.08187108516456588, 
 NS cell match prob = 0.010476167565063825, XF cell match prob = 0.010476167565063825


Trained model:

In [31]:
file = 'drive/MyDrive/Colab_Notebooks/CPSC532S/final_project/model/trained_model_semihard_allcells_alpha' + \
        str(config.ALPHA) + '_margin' + str(config.MARGIN) + '_nchannels' + str(config.N_CHANNELS) + '_15epochs'
model.load_state_dict(torch.load(file, map_location=torch.device('cpu')))
loss_cross, loss_triplet, ct_match_prob, ct_match_prob2, cell_match_prob, cell_match_prob2 = inference(model, data_val)
print('cross entropy loss = {0}, triplet loss = {1}, \n NS cell type match prob = {2}, XF cell type match prob = {3}, \n NS cell match prob = {4}, XF cell match prob = {5}'.format(
    loss_cross,
    loss_triplet, 
    ct_match_prob,
    ct_match_prob2,
    cell_match_prob, 
    cell_match_prob2 
    )
)

cross entropy loss = 3.7436767154269748, triplet loss = 1.0041901109710571, 
 NS cell type match prob = 0.13642621501570656, XF cell type match prob = 0.3571940187423948, 
 NS cell match prob = 0.02466724319235673, XF cell match prob = 0.02466724319235673
